In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyodbc
from datetime import datetime
import os
import glob

In [2]:
#os.chdir("dataPruebaDataEngineer/")
os.chdir('C:/Users/Felipe/Downloads/Pragma/dataPruebaDataEngineer')#cambiar nombre
extension = 'csv'
todos_los_archivos = [i for i in glob.glob('2012*.{}'.format(extension))]
#combina todos los archivos de la lista
df = pd.concat([pd.read_csv(f) for f in todos_los_archivos ])


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 0 to 30
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  143 non-null    object 
 1   price      139 non-null    float64
 2   user_id    143 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 4.5+ KB


In [4]:
#df = pd.read_csv('dataPruebaDataEngineer/2012-1.csv')

df['timestamp'] = pd.to_datetime(df['timestamp'])
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d')
df['price'] = df.price.astype(float)
df = df.replace({np.nan:None}) #hay valores que vienen en blanco se reemplaza por 0

In [5]:
conn_str = (
    r'Driver=ODBC Driver 18 for SQL Server;'
    r'Server=localhost;'
    r'Database=DB_FS_2024;'
    r'Trusted_Connection=yes;'
    r'TrustServerCertificate=YES'
    )
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()
cursor.fast_executemany = True

In [6]:
query = "SELECT * FROM [DB_FS_2024].[dbo].[tbl_flat_files_test];"

df_existing_tables = pd.read_sql(query, cnxn)
df_existing_tables

C:\Users\la carenalgish\AppData\Local\Temp\ipykernel_13816\2732966872.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_existing_tables = pd.read_sql(query, cnxn)


,date_or,price,user_id


In [7]:
new_df =  pd.DataFrame(columns=['timestamp','price','user_id'])
str_df =  pd.DataFrame(columns=['DB_count','row_max','row_min','row_avg'])

In [8]:
for index,row in df.iterrows():
    cursor.execute('INSERT INTO dbo.tbl_flat_files_test([date_or],[price],[user_id]) VALUES (?,?,?)', 
                    row['timestamp'], 
                    row['price'], 
                    row['user_id'])
        # create a new row
    new_row = {'timestamp':row['timestamp'], 'price':row['price'],'user_id':row['user_id']}
    new_row = pd.DataFrame([new_row]) 
    new_df = pd.concat([new_row,new_df], ignore_index=True)
    new_std_row = {'DB_count':len(new_df),'row_max':new_df.max()['price'], 'row_min':new_df.min()['price'],'row_avg':new_df.loc[:,'price'].mean()}
    new_std_row = pd.DataFrame([new_std_row]) 
    str_df = pd.concat([str_df,new_std_row], ignore_index=True)
    cnxn.commit()
    #
    #
    #str_df
cursor.close()
cnxn.close()


In [9]:
str_df.head(100)

,DB_count,row_max,row_min,row_avg
0,1,50.0,50.0,50.000000
1,2,87.0,50.0,68.500000
2,3,87.0,50.0,67.000000
3,4,87.0,20.0,55.250000
4,5,87.0,14.0,47.000000
...,...,...,...,...
95,96,100.0,10.0,57.840426
96,97,100.0,10.0,58.252632
97,98,100.0,10.0,58.583333
98,99,100.0,10.0,58.583333


In [10]:
str_df.to_excel('log.xlsx', index=False)

In [11]:
conn_str = (
    r'Driver=ODBC Driver 18 for SQL Server;'
    r'Server=localhost;'
    r'Database=DB_FS_2024;'
    r'Trusted_Connection=yes;'
    r'TrustServerCertificate=YES'
    )
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()
cursor.fast_executemany = True

In [12]:
validation = pd.read_csv('validation.csv')

In [13]:
for index,row in validation.iterrows():
    cursor.execute('INSERT INTO dbo.tbl_flat_files_test([date_or],[price],[user_id]) VALUES (?,?,?)', 
                    row['timestamp'], 
                    row['price'], 
                    row['user_id'])
        # create a new row
    new_row = {'timestamp':row['timestamp'], 'price':row['price'],'user_id':row['user_id']}
    new_row = pd.DataFrame([new_row]) 
    new_df = pd.concat([new_row,new_df], ignore_index=True)
    new_std_row = {'DB_count':len(new_df),'row_max':new_df.max()['price'], 'row_min':new_df.min()['price'],'row_avg':new_df.loc[:,'price'].mean()}
    new_std_row = pd.DataFrame([new_std_row]) 
    str_df = pd.concat([str_df,new_std_row], ignore_index=True)
    cnxn.commit()
    #
    #
    #str_df
cursor.close()
cnxn.close()


In [14]:
str_df

,DB_count,row_max,row_min,row_avg
0,1,50.0,50.0,50.000000
1,2,87.0,50.0,68.500000
2,3,87.0,50.0,67.000000
3,4,87.0,20.0,55.250000
4,5,87.0,14.0,47.000000
...,...,...,...,...
146,147,100.0,10.0,57.258741
147,148,100.0,10.0,57.076389
148,149,100.0,10.0,57.110345
149,150,100.0,10.0,56.808219


In [15]:
str_df.to_excel('log_validation.xlsx', index=False)